# 🌞🔬 Análisis Exploratorio HelioBio-Economic
## Investigación de Correlaciones Solares-Económicas

**Autor:** Benjamin Cabeza Durán (mechmind-dwv)  
**Asistente:** DeepSeek AI  
**Fecha:** Generado automáticamente

---

Este notebook realiza análisis exploratorio de datos para investigar las correlaciones entre actividad solar y ciclos económicos, extendiendo el trabajo pionero de Alexander Chizhevsky.

In [ ]:
# Configuración inicial
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import sys
import os

# Añadir path para imports del proyecto
sys.path.append('../')

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Módulos importados correctamente")

## 1. Carga y Preparación de Datos

In [ ]:
from app.services.nasa_solar_service import NASASolarService
from app.services.economic_data_service import EconomicDataService
from app.services.correlation_service import CorrelationService

# Inicializar servicios
nasa_service = NASASolarService()
economic_service = EconomicDataService()
correlation_service = CorrelationService()

# Cargar datos históricos
print("📥 Cargando datos históricos...")

# Datos solares (50 años)
solar_data = await nasa_service.get_historical_solar_data(50)
print(f"🌞 Datos solares: {len(solar_data)} registros")

# Datos económicos
economic_data = await economic_service.get_long_term_economic_data()
print(f"💹 Datos económicos: {len(economic_data)} registros")

# Datos de mercado
market_data = await economic_service.get_market_data("^GSPC", "50y")
sp500_prices = [item['price'] for item in market_data['market_data']]
sp500_dates = [item['timestamp'] for item in market_data['market_data']]
sp500_series = pd.Series(sp500_prices, index=pd.to_datetime(sp500_dates))

print("✅ Datos cargados correctamente")

## 2. Análisis Exploratorio de Series Temporales

In [ ]:
# Visualización de series temporales
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Manchas solares
if 'sunspots' in solar_data.columns:
    axes[0, 0].plot(solar_data.index, solar_data['sunspots'], color='orange', linewidth=1)
    axes[0, 0].set_title('Manchas Solares - Serie Temporal', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Número de Manchas')
    axes[0, 0].grid(True, alpha=0.3)

# S&P 500
axes[0, 1].plot(sp500_series.index, sp500_series.values, color='green', linewidth=1)
axes[0, 1].set_title('S&P 500 - Serie Temporal', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Precio (USD)')
axes[0, 1].grid(True, alpha=0.3)

# Flujo solar
if 'solar_flux' in solar_data.columns:
    axes[1, 0].plot(solar_data.index, solar_data['solar_flux'], color='red', linewidth=1)
    axes[1, 0].set_title('Flujo Solar - Serie Temporal', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('Flux (sfu)')
    axes[1, 0].grid(True, alpha=0.3)

# GDP (si está disponible)
if 'GDP' in economic_data.columns:
    axes[1, 1].plot(economic_data.index, economic_data['GDP'], color='blue', linewidth=1)
    axes[1, 1].set_title('GDP - Serie Temporal', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('GDP (Billones USD)')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Análisis de Ciclos y Estacionalidad

In [ ]:
from scipy import signal
from scipy.fft import fft, fftfreq

# Análisis espectral de manchas solares
if 'sunspots' in solar_data.columns:
    sunspots_clean = solar_data['sunspots'].dropna()
    
    # FFT para detección de ciclos
    n = len(sunspots_clean)
    yf = fft(sunspots_clean.values)
    xf = fftfreq(n, 1/12)  # Datos mensuales
    
    # Encontrar frecuencias dominantes
    idx = np.where(xf > 0)  # Frecuencias positivas
    frequencies = xf[idx]
    power = np.abs(yf[idx])
    
    # Convertir a períodos en años
    periods = 1 / frequencies
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Serie temporal con media móvil
    ax1.plot(sunspots_clean.index, sunspots_clean.values, alpha=0.7, label='Original')
    ax1.plot(sunspots_clean.index, sunspots_clean.rolling(132).mean(), 
             color='red', linewidth=2, label='Media móvil 11 años')
    ax1.set_title('Ciclos Solares - Manchas Solares', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Número de Manchas')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Espectro de potencia
    ax2.plot(periods, power, color='purple')
    ax2.set_xlim(0, 20)  # Focus en ciclos hasta 20 años
    ax2.set_title('Espectro de Potencia - Ciclos Solares', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Período (años)')
    ax2.set_ylabel('Potencia')
    ax2.axvline(11, color='red', linestyle='--', alpha=0.7, label='Ciclo Schwabe (11 años)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Identificar ciclos dominantes
    dominant_idx = np.argsort(power)[-5:]  # Top 5 frecuencias
    print("🔍 Ciclos dominantes detectados:")
    for idx in dominant_idx:
        if periods[idx] <= 20:  # Filtrar ciclos razonables
            print(f"  - {periods[idx]:.2f} años (potencia: {power[idx]:.2f})")

## 4. Correlación entre Variables Solares y Económicas

In [ ]:
# Preparar datos para correlación
if 'sunspots' in solar_data.columns and not sp500_series.empty:
    # Alinear fechas
    common_dates = solar_data.index.intersection(sp500_series.index)
    sunspots_aligned = solar_data.loc[common_dates, 'sunspots']
    sp500_aligned = sp500_series.loc[common_dates]
    
    # Remover tendencias
    sunspots_detrended = sunspots_aligned - sunspots_aligned.rolling(12).mean()
    sp500_detrended = sp500_aligned - sp500_aligned.rolling(12).mean()
    
    # Calcular correlación con diferentes lags
    max_lag = 60  # 5 años
    correlations = []
    lags = range(-max_lag, max_lag + 1)
    
    for lag in lags:
        if lag < 0:
            corr = np.corrcoef(sunspots_aligned.iloc[:lag].values, 
                             sp500_aligned.iloc[-lag:].values)[0, 1]
        elif lag > 0:
            corr = np.corrcoef(sunspots_aligned.iloc[lag:].values, 
                             sp500_aligned.iloc[:-lag].values)[0, 1]
        else:
            corr = np.corrcoef(sunspots_aligned.values, sp500_aligned.values)[0, 1]
        correlations.append(corr)
    
    # Visualizar correlación cruzada
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Serie temporal comparativa
    ax1.plot(sunspots_detrended.index, sunspots_detrended.values, 
             color='orange', label='Manchas Solares (detrended)', alpha=0.7)
    ax1.plot(sp500_detrended.index, sp500_detrended.values, 
             color='green', label='S&P 500 (detrended)', alpha=0.7)
    ax1.set_title('Series Temporales Alineadas (Detrended)', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Correlación cruzada
    ax2.plot(lags, correlations, color='blue', linewidth=2)
    ax2.axvline(0, color='red', linestyle='--', alpha=0.5, label='Lag 0')
    ax2.set_title('Correlación Cruzada: Manchas Solares vs S&P 500', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Lag (meses)')
    ax2.set_ylabel('Correlación')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Encontrar lag óptimo
    optimal_lag = lags[np.argmax(np.abs(correlations))]
    max_correlation = correlations[np.argmax(np.abs(correlations))]
    
    print(f"🔗 Correlación máxima: {max_correlation:.3f} en lag {optimal_lag} meses")
    print(f"📊 Correlación contemporánea (lag 0): {correlations[max_lag]:.3f}")

## 5. Análisis de Crisis Históricas vs Ciclos Solares

In [ ]:
from app.core.economic_cycles import economic_cycle_analyzer

# Análisis de crisis históricas
crisis_analysis = economic_cycle_analyzer.analyze_historical_crises()

# Visualizar crisis en contexto de ciclos solares
if 'sunspots' in solar_data.columns:
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Manchas solares
    ax.plot(solar_data.index, solar_data['sunspots'], 
            color='orange', linewidth=1, alpha=0.7, label='Manchas Solares')
    
    # Marcar crisis históricas
    crisis_dates = {
        'Gran Depresión': '1929-10-29',
        'Crisis Petróleo': '1973-10-16', 
        'Lunes Negro': '1987-10-19',
        'Burbuja Dot-com': '2000-03-10',
        'Crisis 2008': '2008-09-15',
        'COVID-19': '2020-03-23'
    }
    
    for crisis, date_str in crisis_dates.items():
        date = pd.to_datetime(date_str)
        if date in solar_data.index:
            crisis_value = solar_data.loc[date, 'sunspots']
            ax.scatter(date, crisis_value, color='red', s=100, zorder=5)
            ax.annotate(crisis, (date, crisis_value), 
                       xytext=(10, 10), textcoords='offset points',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    ax.set_title('Crisis Históricas en Contexto de Ciclos Solares', fontsize=16, fontweight='bold')
    ax.set_ylabel('Número de Manchas Solares')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Análisis estadístico
    print("📈 Análisis de Crisis vs Fases Solares:")
    print(f"  - Crisis en máximos solares: {crisis_analysis['correlation_metrics']['crisis_at_solar_maximum']:.1%}")
    print(f"  - Crisis en mínimos solares: {crisis_analysis['correlation_metrics']['crisis_at_solar_minimum']:.1%}")
    print(f"  - Correlación general: {crisis_analysis['correlation_metrics']['overall_correlation']:.1%}")

## 6. Heatmap de Correlaciones Multivariadas

In [ ]:
# Preparar matriz de correlación
correlation_data = {}

if 'sunspots' in solar_data.columns:
    correlation_data['Sunspots'] = solar_data['sunspots']
if 'solar_flux' in solar_data.columns:
    correlation_data['Solar_Flux'] = solar_data['solar_flux']

correlation_data['SP500'] = sp500_series

if 'GDP' in economic_data.columns:
    correlation_data['GDP'] = economic_data['GDP']
if 'UNEMPLOYMENT' in economic_data.columns:
    correlation_data['Unemployment'] = economic_data['UNEMPLOYMENT']

# Crear DataFrame de correlación
corr_df = pd.DataFrame(correlation_data)
corr_df = corr_df.ffill().bfill()  # Limpiar datos

# Matriz de correlación
correlation_matrix = corr_df.corr()

# Visualizar heatmap
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Máscara para triángulo superior

sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            square=True,
            fmt='.3f',
            cbar_kws={"shrink": .8})

plt.title('Matriz de Correlación: Variables Solares vs Económicas', 
          fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("🔍 Análisis de Correlaciones Clave:")
key_pairs = [('Sunspots', 'SP500'), ('Solar_Flux', 'GDP'), ('Sunspots', 'Unemployment')]
for var1, var2 in key_pairs:
    if var1 in correlation_matrix.columns and var2 in correlation_matrix.columns:
        corr_value = correlation_matrix.loc[var1, var2]
        strength = "Fuerte" if abs(corr_value) > 0.5 else "Moderada" if abs(corr_value) > 0.3 else "Débil"
        direction = "positiva" if corr_value > 0 else "negativa"
        print(f"  - {var1} vs {var2}: {corr_value:.3f} ({strength} {direction})")

## 7. Análisis de Ondas Largas (Kondratiev)

In [ ]:
from app.core.kondratiev_analysis import kondratiev_analyzer

# Análisis de ondas largas
kondratiev_analysis = kondratiev_analyzer.analyze_long_waves()
kondratiev_report = kondratiev_analyzer.generate_kondratiev_report()

print("🌊 ANÁLISIS DE ONDAS LARGAS KONDRATIEV")
print("=" * 50)
print(f"Onda Actual: #{kondratiev_analysis.current_wave.wave_number}")
print(f"Paradigma Tecnológico: {kondratiev_analysis.current_wave.technological_paradigm}")
print(f"Fase Actual: {kondratiev_analysis.current_phase.value}")
print(f"Progreso de Fase: {kondratiev_analysis.phase_progress:.1%}")
print(f"Próxima Transición: {kondratiev_analysis.next_phase_transition.strftime('%Y-%m')}")
print()

print("🔗 Sincronización Solar:")
for metric, value in kondratiev_analysis.solar_correlation.items():
    print(f"  - {metric}: {value:.3f}")
print()

print("📈 Implicaciones Económicas:")
for key, value in kondratiev_analysis.economic_implications.items():
    if isinstance(value, list) and value:
        print(f"  - {key}:")
        for item in value[:3]:  # Mostrar primeros 3 items
            print(f"    • {item}")
    elif not isinstance(value, list):
        print(f"  - {key}: {value}")
print()

print("⚠️ Evaluación de Riesgos:")
for key, value in kondratiev_analysis.risk_assessment.items():
    if not isinstance(value, list):
        print(f"  - {key}: {value}")

## 8. Conclusiones y Hallazgos Preliminares

In [ ]:
print("🎯 CONCLUSIONES DEL ANÁLISIS EXPLORATORIO")
print("=" * 50)

hallazgos = []

# Hallazgo 1: Ciclos solares
if 'sunspots' in solar_data.columns:
    ciclo_promedio = solar_data['sunspots'].rolling(132).mean().std()
    hallazgos.append(f"• Ciclo solar de ~11 años confirmado (variabilidad: {ciclo_promedio:.1f} manchas)")

# Hallazgo 2: Correlación solar-económica
if 'sunspots' in solar_data.columns and not sp500_series.empty:
    corr_contemporanea = np.corrcoef(solar_data['sunspots'].dropna().values, 
                                   sp500_series.dropna().values)[0, 1]
    hallazgos.append(f"• Correlación contemporánea manchas solares-S&P500: {corr_contemporanea:.3f}")

# Hallazgo 3: Crisis y ciclos
crisis_corr = crisis_analysis['correlation_metrics']['overall_correlation']
hallazgos.append(f"• Correlación crisis-fases solares: {crisis_corr:.1%}")

# Hallazgo 4: Kondratiev actual
hallazgos.append(f"• Onda Kondratiev actual: #{kondratiev_analysis.current_wave.wave_number} ({kondratiev_analysis.current_phase.value})")

# Hallazgo 5: Sincronización
sync_strength = kondratiev_analysis.solar_correlation.get('predicted_sync_strength', 0)
hallazgos.append(f"• Sincronización solar-Kondratiev prevista: {sync_strength:.1%}")

print("\n".join(hallazgos))

print("\n🔬 RECOMENDACIONES PARA INVESTIGACIÓN FUTURA:")
recomendaciones = [
    "• Profundizar análisis de causalidad con tests de Granger",
    "• Investigar mecanismos de transmisión solar-económica", 
    "• Extender análisis a otros indicadores económicos",
    "• Validar modelos predictivos con backtesting",
    "• Estudiar efectos regionales diferenciados"
]
print("\n".join(recomendaciones))

print(f"\n📅 Análisis generado: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("🌞 HelioBio-Economic v1.0 - Investigación en curso")